# SetFit for Multilabel Text Classification

In [1]:
epoch = 1

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:

model_id = "sentence-transformers/all-MiniLM-L6-v2"

In [5]:
from datasets import load_dataset
import pandas as pd
import os
import glob

#change when applied for summaries. 
#note that the datasets contain different themes
test = pd.read_csv('test_sub.csv')
train = pd.read_csv('train_sub.csv')


In [6]:
from datasets import Dataset
train = Dataset.from_pandas(train)
test = Dataset.from_pandas(test)

In [7]:
# check the themes & length test set 
test

Dataset({
    features: ['husband and wife', 'murder', 'the desire for vengeance', 'infatuation', 'romantic love', 'friendship', 'spouse murder', 'greed for riches', 'father and son', 'extramarital affair', 'content'],
    num_rows: 192
})

In [8]:
# check the themes & length train set 
train

Dataset({
    features: ['husband and wife', 'murder', 'the desire for vengeance', 'infatuation', 'romantic love', 'friendship', 'spouse murder', 'greed for riches', 'father and son', 'extramarital affair', 'content'],
    num_rows: 764
})

In [9]:
import numpy as np

#create list of the target themes
features = train.column_names
features.remove("content")
features

['husband and wife',
 'murder',
 'the desire for vengeance',
 'infatuation',
 'romantic love',
 'friendship',
 'spouse murder',
 'greed for riches',
 'father and son',
 'extramarital affair']

In [10]:
# encode themes in a single 'label' feature
def encode_labels(record):
    return {"labels": [record[feature] for feature in features]}


train = train.map(encode_labels)

Map:   0%|          | 0/764 [00:00<?, ? examples/s]

In [11]:

test = test.map(encode_labels)

Map:   0%|          | 0/192 [00:00<?, ? examples/s]

In [12]:
# finetune setfit model
from setfit import SetFitModel

model = SetFitModel.from_pretrained(model_id, multi_target_strategy="one-vs-rest")

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [13]:
# Training arguments
# change sampling_strategy for 'oversampling', 'unique' and 'undersampling'
from setfit import TrainingArguments
args = TrainingArguments(num_epochs = epoch, sampling_strategy = 'oversampling')

In [14]:
# create trainer
from sentence_transformers.losses import CosineSimilarityLoss
from setfit import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train,
    eval_dataset=test,
    column_mapping={"content": "text", "labels": "label"},
)

Applying column mapping to the training dataset
Applying column mapping to the evaluation dataset


Map:   0%|          | 0/764 [00:00<?, ? examples/s]

In [15]:
# train model
trainer.train()

***** Running training *****
  Num unique pairs = 425640
  Batch size = 16
  Num epochs = 1


Step,Training Loss
1,0.328900
50,0.290600
100,0.262100
150,0.245800
200,0.241200
250,0.238200
300,0.239200
350,0.236600
400,0.239900
450,0.228000


In [16]:
metrics = trainer.evaluate()
metrics

***** Running evaluation *****


{'accuracy': 0.11979166666666667}

In [17]:
# save model
model.save_pretrained("sub_setfit-all-MiniLM-L6-v2_epoch_1_over")

In [18]:
from setfit import SetFitModel
model = SetFitModel.from_pretrained("sub_setfit-all-MiniLM-L6-v2_epoch_1_over") # Load from a local directory

In [19]:
# create csv  containing model predictions
model_np = model.predict(test['content']).numpy()
df = pd.DataFrame(model_np)
df.columns = features

In [20]:
# save predictions as csv
df.to_csv("sub_setfit-all-MiniLM-L6-v2_epoch_1_over.csv")